In [2]:
import numpy as np
import matplotlib.pyplot as plt
#import cma
from es import SimpleGA, CMAES, PEPG, OpenES

def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

def BCE_loss(y,p):
    return np.mean(-y*np.log(p) - (1 - y)*np.log(1 - p))


def binarize(x):
    res = x > 0.5
    return res.astype(int)


def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def relu(x):
  return np.maximum(x, 0)

def passthru(x):
  return x

# useful for discrete actions
def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)

# useful for discrete actions
def sample(p):
  return np.argmax(np.random.multinomial(1, p))


class RNNCell:
  def __init__(self, input_size, weight, bias):
    self.input_size=input_size
    self.weight = weight
    self.bias = bias
  def __call__(self, x, h):
    concat = np.concatenate((x, h), axis=1)
    hidden = np.matmul(concat, self.weight)+self.bias
    return np.tanh(hidden)

class RNNModel:
    def __init__(self):
    

        self.hidden_size = 10

        self.layer_1 = 10
        self.layer_2 = 10

        self.rnn_mode = True

        self.input_size = 1
        self.output_size = 1
        self.alpha = 0.0


        self.shapes = [ (self.input_size + self.hidden_size, 1*self.hidden_size), # RNN weights
                        (self.input_size + self.hidden_size, self.layer_1),# predict actions output
                        (self.layer_1, self.output_size)] # predict actions output

        self.weight = []
        self.bias = []
        self.param_count = 0

        idx = 0
        for shape in self.shapes:
          self.weight.append(np.zeros(shape=shape))
          self.bias.append(np.zeros(shape=shape[1]))
          self.param_count += (np.product(shape) + shape[1])
          idx += 1

        self.init_h = np.zeros((1, self.hidden_size))
        self.h = self.init_h
        self.param_count += 1*self.hidden_size

        self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

    def reset(self):
        self.h = sigmoid(self.init_h)


    def get_action(self, x):
        obs = x.reshape(1, self.input_size)

        # update rnn:
        #update_obs = np.concatenate([obs, action], axis=1)
       
        self.h = sigmoid(self.rnn(x, self.h))

        
        # get action
        x = np.concatenate([x, self.h], axis=1)

        # calculate action using 2 layer network from output
        hidden = np.tanh(np.matmul(x, self.weight[1]) + self.bias[1])
        action = sigmoid(np.matmul(hidden, self.weight[2]) + self.bias[2])

        return action[0]

    def set_model_params(self, model_params):
        pointer = 0
        for i in range(len(self.shapes)):
          w_shape = self.shapes[i]
          b_shape = self.shapes[i][1]
          s_w = np.product(w_shape)
          s = s_w + b_shape
          chunk = np.array(model_params[pointer:pointer+s])
          self.weight[i] = chunk[:s_w].reshape(w_shape)
          self.bias[i] = chunk[s_w:].reshape(b_shape)
          pointer += s
        # rnn states
        s = self.hidden_size
        self.init_h = model_params[pointer:pointer+s].reshape((1, self.hidden_size))
        self.h = self.init_h
        self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

    def load_model(self, filename):
        with open(filename) as f:    
          data = json.load(f)
        print('loading file %s' % (filename))
        self.data = data
        model_params = np.array(data[0]) # assuming other stuff is in data
        self.set_model_params(model_params)

    def get_random_model_params(self, stdev=0.1):
        return np.random.randn(self.param_count)*stdev

    def update_alpha(self):
        self.alpha += 0.001
    

In [3]:


class RNNModel2:
    def __init__(self):
    

        self.hidden_size = 10

        self.layer_1 = 10
        self.layer_2 = 10

        self.rnn_mode = True

        self.input_size = 1
        self.output_size = 10
        self.alpha = 1.0


        self.shapes = [ (self.input_size + self.hidden_size, 1*self.hidden_size), # RNN weights
                        (self.input_size + self.hidden_size, self.layer_1),# predict actions output
                        (self.layer_1, self.output_size)] # predict actions output

        self.weight = []
        self.bias = []
        self.param_count = 0

        idx = 0
        for shape in self.shapes:
          self.weight.append(np.zeros(shape=shape))
          self.bias.append(np.zeros(shape=shape[1]))
          self.param_count += (np.product(shape) + shape[1])
          idx += 1

        self.init_h = np.zeros((1, self.hidden_size))
        self.h = self.init_h
        self.param_count += 1*self.hidden_size

        self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

    def reset(self):
        self.h = self.init_h
        self.h =  (1 - self.alpha)*sigmoid(self.h)+ self.alpha*binarize(sigmoid(self.h))


    def get_action(self, x):
        obs = x.reshape(1, self.input_size)

        # update rnn:
        #update_obs = np.concatenate([obs, action], axis=1)
        h =  (1 - self.alpha)*sigmoid(self.h)+ self.alpha*binarize(sigmoid(self.h))
        self.h = self.rnn(x, h)

        h =  (1 - self.alpha)*sigmoid(self.h)+ self.alpha*binarize(sigmoid(self.h))
        # get action
        x = np.concatenate([x, h], axis=1)

        # calculate action using 2 layer network from output
        hidden = np.tanh(np.matmul(x, self.weight[1]) + self.bias[1])
        action = sigmoid(np.matmul(hidden, self.weight[2]) + self.bias[2])

        return action[0]

    def set_model_params(self, model_params):
        pointer = 0
        for i in range(len(self.shapes)):
          w_shape = self.shapes[i]
          b_shape = self.shapes[i][1]
          s_w = np.product(w_shape)
          s = s_w + b_shape
          chunk = np.array(model_params[pointer:pointer+s])
          self.weight[i] = chunk[:s_w].reshape(w_shape)
          self.bias[i] = chunk[s_w:].reshape(b_shape)
          pointer += s
        # rnn states
        s = self.hidden_size
        self.init_h = model_params[pointer:pointer+s].reshape((1, self.hidden_size))
        self.h = self.init_h
        self.rnn = RNNCell(self.input_size, self.weight[0], self.bias[0])

    def load_model(self, filename):
        with open(filename) as f:    
          data = json.load(f)
        print('loading file %s' % (filename))
        self.data = data
        model_params = np.array(data[0]) # assuming other stuff is in data
        self.set_model_params(model_params)

    def get_random_model_params(self, stdev=0.1):
        return np.random.randn(self.param_count)*stdev

    def update_alpha(self):
        self.alpha += 0.001
    

In [4]:
model = RNNModel()
model2 = RNNModel2()
#model.get_action(np.array([[4]]))


In [5]:

NPARAMS = model.param_count   # make this a 100-dimensinal problem.
NPOPULATION = 410    # use population size of 101.
MAX_ITERATION = 4010 # run each solver for 5000 generations.


In [6]:

def recurrency_label(seq_len):
    
    labels = []
    X = []

    X = np.zeros([seq_len,1])
    #X[0,:] = 1.0
    for ii in range(seq_len):
        if ii % 40 == 0:
            labels.append(np.ones([1,1]))

        else:
            labels.append(np.zeros([1,1]))

    return X, np.concatenate(labels, axis=0)

def evluate_func(data):
    model, params =  data
    model.set_model_params(params)
    model.reset()
    loss_cum = 0
    Xs, labels  = recurrency_label(42)

    for x, label in zip(Xs,labels):
        

        x = np.array([x])
        pred = model.get_action(x)
        
        #print(label, pred)
        loss = BCE_loss(label, pred)
        loss_cum += loss
    #print(loss_cum)
    return -loss_cum

def evluate_func(data):
    model, params =  data
    model.set_model_params(params)
    model.reset()
    loss_cum = 0
    Xs, labels  = recurrency_label(42)

    for x, label in zip(Xs,labels):
        

        x = np.array([x])
        pred = model.get_action(x)
        
        #print(label, pred)
        loss = BCE_loss(label, pred)
        loss_cum += loss
    #print(loss_cum)
    return -loss_cum

In [7]:


# defines genetic algorithm solver
ga = SimpleGA(NPARAMS,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )

In [8]:
import multiprocessing as mp
print(mp.cpu_count())
pool = mp.Pool(mp.cpu_count())
fit_func = evluate_func
# defines a function to use solver to solve fit_func
def test_solver(solver):
    history = []
    j = 0
    while True:
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)
        #print(solutions)
        fitness_list = pool.map(fit_func, [(model,solutions[i]) for i in range(solver.popsize)])
            

        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 10 == 0:
            print("fitness at iteration", (j+1), result[1])
#             print("Best:",solver.elite_rewards[0])   
            
#         if -solver.elite_rewards[0] < 1:
#             model.update_alpha()
#             print('Alpha changed to', model.alpha)
#             for kk in range(len(solver.elite_rewards)):
#                 solver.elite_rewards[kk] = fit_func((model,solver.elite_params[kk]))
#                 solver.forget_best = True
#             else:
#                 solver.forget_best = False
        if -result[1] <= 0.0001:
            print("local optimum discovered by solver:\n", result[0])
            print("fitness score at this local optimum:", result[1])
            return history, result
        j += 1
        
             
    

ga_history, result = test_solver(ga)

32
fitness at iteration 10 -4.938518463346954
fitness at iteration 20 -4.531225489333141
fitness at iteration 30 -4.144667134398752


Process ForkPoolWorker-22:
Process ForkPoolWorker-26:
Process ForkPoolWorker-30:
Process ForkPoolWorker-32:
Process ForkPoolWorker-20:
Process ForkPoolWorker-6:
Process ForkPoolWorker-28:
Process ForkPoolWorker-1:
Process ForkPoolWorker-31:
Process ForkPoolWorker-3:
Process ForkPoolWorker-14:
Process ForkPoolWorker-9:
Process ForkPoolWorker-19:
Process ForkPoolWorker-24:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-21:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-15:
Traceback (most recent call last):
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/mu

  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/shared/gabriele/miniconda/envs/ARC/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
Process 

In [ ]:
def MSE_loss(y,p):
    
    return np.mean(np.abs(y - p))


def evluate_func2(data):
    model, target_model, params =  data
    model.set_model_params(params)
    model.reset()
    target_model.reset()
    loss_cum = 0
    Xs, labels  = recurrency_label(42)

    for x, label in zip(Xs,labels):
        

        x = np.array([x])
        ac = target_model.get_action(x)
        pred = model.get_action(x)
        
        
        #print(label, pred)
        loss = MSE_loss(target_model.h, pred)
#         print(target_model.h, pred)
        loss_cum += loss
    #print(loss_cum)
    return -loss_cum

ga = SimpleGA(model2.param_count,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )


print(mp.cpu_count())


pool = mp.Pool(mp.cpu_count())
fit_func = evluate_func2
# defines a function to use solver to solve fit_func
def test_solver(solver, target_model):
    history = []
    j = 0
    while True:
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)

        fitness_list = pool.map(fit_func, [(model2, target_model, solutions[i]) for i in range(solver.popsize)])
            

        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 10 == 0:
            print("fitness at iteration", (j+1), result[1])
        

        if result[1] >= 0.0001:
            break
        #j += 1
        
             
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
    return history, result

model.set_model_params(result[0])
target_model = model
#print(target_model)
ga_history, result = test_solver(ga, target_model)

32


In [ ]:
len(ga.elite_params)
ga.elite_rewards

In [23]:

def evluate_func(model, target_model, params):
#     model, target_model, params =  data
    model.set_model_params(params)
    model.reset()
    target_model.reset()
    loss_cum = 0
    Xs, labels  = recurrency_label(42)

    for x, label in zip(Xs,labels):
        

        x = np.array([x])
        ac = target_model.get_action(x)
        pred = model.get_action(x)
        
        #print(binarize(model.h))
        
        #print(label, pred)
        loss = BCE_loss(target_model.h, pred)
        print(target_model.h[0,3],pred[6])
        loss_cum += loss
    #print(loss_cum)
    return -loss_cum
evluate_func(model2,target_model, result[0])

IndexError: tuple index out of range

In [9]:
cmaes = CMAES(NPARAMS,
              popsize=NPOPULATION,
              weight_decay=0.0,
              sigma_init = 2.0
          )
cma_history = test_solver(cmaes)

In [11]:
# defines PEPG (NES) solver
pepg = PEPG(NPARAMS,                         # number of model parameters
            sigma_init=0.5,                  # initial standard deviation
            learning_rate=0.01,               # learning rate for standard deviation
            learning_rate_decay=1.0,       # don't anneal the learning rate
            popsize=NPOPULATION,             # population size
            average_baseline=False,          # set baseline to average of batch
            weight_decay=0.00,            # weight decay coefficient
            rank_fitness=False,           # use rank rather than fitness numbers
            forget_best=False)   
pepg_history = test_solver(pepg)

In [ ]:
solutions = ga.ask()
evluate_func(model, solutions[0])

In [165]:
X = model.forward(np.ones([1,12]))
#model.forward(X[0])
X.shape

(1, 11)

In [49]:
model.lyr_shapes

[array([20, 12]), array([20, 20]), array([11, 20])]

In [50]:
model.num_params()

860